# 🚀 Requête WIKIDATA

## 📑 Mode d'emploi

Suivre les instructions au fil du notebook et exécuter une à une les cellules de code en appuyant sur la petite flèche à gauche (▶️)

## 🔨 Construction de l'environnement nécessaire et configuration

### Installation des modules

In [2]:
# 📦 MODULES NECESSAIRES : NORMALEMENT, NE RUN QU'A LA PREMIERE UTILISATION
%pip install -q SPARQLWrapper tqdm pandas

print("✅ Installation terminée !")

Note: you may need to restart the kernel to use updated packages.
✅ Installation terminée !



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Configuration


#### Paramètres

In [3]:
# 🔧 IMPORTS PYTHON
import json
import csv
import time
import re
import os
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
import pandas as pd

print("🔧 Imports terminés !")

🔧 Imports terminés !


In [4]:
# 🔧 CONFIGURATION PERSONNALISABLE DE LA REQUETE

WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql" # Endpoint SPARQL de Wikidata
RATE_LIMIT_DELAY = 3.0  # Délai entre les requêtes
BATCH_SIZE = 10  # Taille des batches pour les requêtes
MAX_RETRIES = 3  # Nombre maximal de tentatives en cas d'échec
REQUEST_TIMEOUT = 60 # Temps au bout duquel un requête s'arrête automatiquement s'il n'y a pas de réponse (en secondes)
ENRICHMENT_BATCH_SIZE = 15 # Taille du batch pour l'enrichissement des données
LOOP_LIMIT = 100 # Nombre réponses limite par boucle (permet de requêter petit à petit pour ne pas surcharger l'API)
LOOP_OFFSET = 0 # Décalage pour la pagination des résultats
MAX_RESULTS = None  # Nombre maximal de résultats à récupérer (pour éviter de surcharger l'API)


# 📁 Configuration du dossier de sortie 
output_dir = "./output" # Paramètre à remplacer si vous souhaitez un autre dossier de sortie (faire un copier coller du chemin d'un fichier)
os.makedirs(output_dir, exist_ok=True)

print("="*100)
print("🚀 CONFIGURATION TERMINÉE".center(100))
print("="*100)
print(
    f"📁 Dossier de sortie                  {output_dir}\n"
    f"⏱️  Rate limit                        {RATE_LIMIT_DELAY}s entre requêtes\n"
    f"📦 Taille des batches                 {BATCH_SIZE}\n"
    f"🔄 Nombre maximal de tentatives       {MAX_RETRIES}\n"
    f"⏳ Délai de timeout des requêtes      {REQUEST_TIMEOUT}s\n"
    f"🔍 Taille du batch d'enrich.         {ENRICHMENT_BATCH_SIZE}\n"
    f"🔁 Limite de boucle                  {LOOP_LIMIT} itérations\n"
    f"🌐 Endpoint Wikidata                 {WIKIDATA_ENDPOINT}"
)
print("="*100)


                                      🚀 CONFIGURATION TERMINÉE                                      
📁 Dossier de sortie                  ./output
⏱️  Rate limit                        3.0s entre requêtes
📦 Taille des batches                 10
🔄 Nombre maximal de tentatives       3
⏳ Délai de timeout des requêtes      60s
🔍 Taille du batch d'enrich.         15
🔁 Limite de boucle                  100 itérations
🌐 Endpoint Wikidata                 https://query.wikidata.org/sparql


#### Client sparql

In [5]:
# 🛠️ FONCTION POUR CHOISIR UN TERME A REQUÊTER

def ask_query_term():
    """
    Fonction pour demander à l'utilisateur de choisir entre recherche par id ou par label
    Et lui demande de rentrer la valeur à rechercher
    """
    
    print("🔍 Choisissez le type de recherche :")
    print("1. Recherche par ID (ex: Q42)")
    print("2. Recherche par label (ex: Douglas Adams)")
    
    choice = input("Entrez 1 ou 2 : ").strip()
    
    if choice == "2":
        search_term = input("Entrez le nom de l'entité à rechercher dans wikidata, en anglais (ex: aeronautics) : ").strip()
        if not search_term:
            print("❌ Aucun nom d'entité fourni.")
            exit(1)
        return f'"{search_term}"'  # Guillemets pour recherche textuelle
    elif choice == "1":
        search_term = input("Entrez l'ID de l'entité à rechercher dans wikidata (ex: Q42) : ").strip()
        # Validation de l'ID de l'entité
        if not search_term or not re.match(r"^Q\d+$", search_term):
            print("❌ ID d'entité invalide. Veuillez entrer un ID valide (ex: Q42).")
            exit(1)
        return search_term  # ← Retourner juste l'ID sans préfixe
    else:
        print("Choix invalide, veuillez réessayer.")
        return ask_query_term()

# 🛠️ FONCTIONS DE PARAMETRAGE DES REQUÊTES ET DU CARNET

################################################################################
# FONCTION POUR CRÉER UN CLIENT SPARQL
################################################################################
def create_sparql_client():
    """
    Crée un client SPARQL pour interagir avec Wikidata
    :return: Instance de SPARQLWrapper configurée pour Wikidata
    """
    sparql = SPARQLWrapper(WIKIDATA_ENDPOINT)
    sparql.setReturnFormat(JSON)
    sparql.setTimeout(REQUEST_TIMEOUT)
    return sparql

################################################################################
# FONCTION PRINCIPALE POUR EXÉCUTER UNE REQUÊTE SPARQL
################################################################################
def execute_sparql_query(query, max_retries=MAX_RETRIES, use_pagination=False, limit=None, max_results=None):
    """
    Exécute une requête SPARQL avec gestion des erreurs, rate limiting et pagination optionnelle
    :param query: La requête SPARQL à exécuter
    :param max_retries: Nombre maximum de tentatives en cas d'échec
    :param use_pagination: Si True, active la pagination automatique
    :param limit: Taille des pages pour la pagination (défaut: LOOP_LIMIT)
    :param max_results: Nombre maximum de résultats à récupérer (None = illimité)
    :return: Résultats de la requête ou une liste vide en cas d'échec
    """
    sparql = create_sparql_client()
    print(f"🔍 Exécution de la requête SPARQL :\n{query}\n")
    
    # MODE SIMPLE SANS PAGINATION
    if not use_pagination:
        print("🚀 Mode simple (sans pagination) - Envoi de la requête...")
        for attempt in range(max_retries):
            try:
                print(f"📡 Tentative {attempt + 1}/{max_retries} - Envoi de la requête...")
                sparql.setQuery(query)
                
                print("⏳ Attente de la réponse du serveur...")
                results = sparql.query().convert()
                
                result_count = len(results["results"]["bindings"])
                print(f"✅ Requête réussie ! {result_count} résultats obtenus")
                
                
                return results["results"]["bindings"]
                
            except Exception as e:
                print(f"⚠️  Tentative {attempt + 1}/{max_retries} échouée: {e}")
                if attempt < max_retries - 1:
                    wait_time = RATE_LIMIT_DELAY * (attempt + 2)
                    print(f"⏳ Attente de {wait_time}s avant nouvelle tentative...")
                    time.sleep(wait_time)
                else:
                    print(f"❌ Requête échouée après {max_retries} tentatives")
                    return []
        return []
    
    # MODE PAGINATION ACTIVE
    if limit is None:
        limit = LOOP_LIMIT
    
    all_results = []
    offset = 0
    
    print(f"🔍 Début de la pagination (limit={limit})...")
    
    while True:
        paginated_query = f"{query.rstrip()} LIMIT {limit} OFFSET {offset}"
        print(f"🔹 Requête OFFSET {offset}, LIMIT {limit}")
        success = False
        bindings = []
        for attempt in range(max_retries):
            try:
                sparql.setQuery(paginated_query)
                results = sparql.query().convert()
                bindings = results["results"]["bindings"]
                success = True
                break
            except Exception as e:
                print(f"⚠️  Tentative {attempt + 1}/{max_retries} échouée à l'offset {offset}: {e}")
                if attempt < max_retries - 1:
                    time.sleep(RATE_LIMIT_DELAY * (attempt + 2))
                else:
                    print(f"❌ Requête échouée après {max_retries} tentatives à l'offset {offset}")
                    return all_results  # Retourner ce qu'on a réussi à récupérer

        if not success:
            break

        if not bindings:
            print("✅ Fin de la pagination - Aucun résultat supplémentaire.")
            break

        all_results.extend(bindings)
        print(f"✅ Récupéré {len(bindings)} résultats (total: {len(all_results)})")
        
        if max_results and len(all_results) >= max_results:
            print(f"🎯 Limite de {max_results} résultats atteinte.")
            all_results = all_results[:max_results]
            break

        offset += limit
        print(f"⏳ Attente de {RATE_LIMIT_DELAY}s...")
        time.sleep(RATE_LIMIT_DELAY)
    
    print(f"🎯 Total final récupéré : {len(all_results)} résultats.")
    return all_results

################################################################################
# FONCTION UTILITAIRE POUR EXTRAIRE LES ID DE WIKIDATA
################################################################################
def clean_entity_id(entity_uri):
    """
    Extrait l'ID d'une entité à partir de son URI
    :param entity_uri: URI de l'entité (ex: "http://www.wikidata.org/entity/Q42'")
    :return: ID de l'entité (ex: "Q42") ou une chaîne vide si l'URI est vide
    """
    if not entity_uri:
        return ""
    return entity_uri.split("/")[-1] if "/" in entity_uri else entity_uri

################################################################################
# FONCTION POUR EXÉCUTER DES REQUÊTES SPARQL EN BATCH : VERIFIER L'UTILITE
################################################################################
def execute_batch_queries(queries, description="Requêtes", use_pagination=False):
    """
    Exécute une liste de requêtes SPARQL en batch
    :param queries: Requête SPARQL unique ou liste de requêtes
    :param description: Description de la tâche pour le logging
    :param use_pagination: Si True, active la pagination pour chaque requête
    :return: Liste de tous les résultats combinés
    """
    # Vérifier si queries est une string ou une liste
    if isinstance(queries, str):
        # Si c'est une string, c'est une seule requête
        print(f"🔹 Exécution d'une requête unique: {description}")
        return execute_sparql_query(queries, use_pagination=use_pagination)
    
    # Si c'est une liste, traiter comme batch
    all_results = []
    for i, query in enumerate(tqdm(queries, desc=description)):
        results = execute_sparql_query(query, use_pagination=use_pagination)
        all_results.extend(results)
        if (i + 1) % BATCH_SIZE == 0:
            time.sleep(RATE_LIMIT_DELAY)
    return all_results

################################################################################
# FONCTION POUR EXÉCUTER UNE REQUÊTE SPARQL AVEC PAGINATION
################################################################################
def execute_paginated_query(base_query, limit=None, max_results=MAX_RESULTS, ask_term=False):
    """
    Fonction helper pour exécuter facilement une requête avec pagination
    """
    
    # Si ask_term=True, on demande le terme à l'utilisateur
    if ask_term:
        search_term = ask_query_term()
        print(f"🔍 Recherche pour le terme : {search_term}")
        if search_term:
            # Déterminer le type de recherche et formater correctement
            if search_term.startswith('Q'):
                # Recherche par ID - ajouter le préfixe wd:
                formatted_term = f"wd:{search_term}"
            else:
                # Recherche textuelle - utiliser tel quel
                formatted_term = search_term
            
            # Remplacer {{search_term}} dans la requête
            base_query = base_query.replace("{{search_term}}", formatted_term)
        else:
            print("❌ Aucun terme fourni, abandon de la requête.")
            return []
    
    return execute_sparql_query(
        base_query, 
        use_pagination=True, 
        limit=limit, 
        max_results=max_results
    )
print("✅ Fonctions de requête SPARQL prêtes !")

✅ Fonctions de requête SPARQL prêtes !


#### Outils Python

In [6]:
# DEFINITION DES TERMES A CHERCHER
# =================================================================================

#search_term = None  # Initialisation de la variable pour le terme de recherche

# def ask_query_term():
#     """
#     Fonction pour demander à l'utilisateur de choisir entre recherche par id ou par label
#     Et lui demande de rentrer la valeur à rechercher
#     """
#     print("🔍 Choisissez le type de recherche :")
#     print("1. Recherche par ID (ex: Q42)")
#     print("2. Recherche par label (ex: Douglas Adams)")
    
#     choice = input("Entrez 1 ou 2 : ").strip()
    
#     if choice == "2":
#         search_term = input("Entrez le nom de l'entité à rechercher dans wikidata, en anglais (ex: aeronautics) : ").strip()
#         if not search_term:
#             print("❌ Aucun nom d'entité fourni.")
#             exit(1)  # Sortir du script si aucun nom n'est fourni
#         return search_term 
#     elif choice == "1":
#         search_term = input("Entrez l'ID de l'entité à rechercher dans wikidata (ex: Q42) : ").strip()
#         # Validation de l'ID de l'entité
#         if not search_term or not re.match(r"^Q\d+$", search_term):
#             print("❌ ID d'entité invalide. Veuillez entrer un ID valide (ex: Q42).")
#             exit(1)  # Sortir du script si l'ID est invalide
#         return search_term
#     else:
#         print("Choix invalide, veuillez réessayer.")
#         return ask_query_term()  # Redemander si le choix est invalide
    
# ask_query_term()

## 🧐 REQUÊTES

Résumé des requêtes disponibles : 
| Type de requête | Résumé | Output |
| --- | --- | --- |
| Recherche par nom |  `SELECT ?item ?itemLabel WHERE { ?item rdfs:label ?itemLabel. FILTER(LANG(?itemLabel) = "en"). FILTER(CONTAINS(LCASE(?itemLabel), "{search_entity_name}")). }` | Label, ID




### 📖 Bibliothèque de requêtes

In [8]:
# =================================================================================
# DEFINITION DE REQUÊTE SPARQL POUR RECHERCHER UNE ENTITÉ PAR NOM
# =================================================================================

# REQUETE DANS LES LABELS
query_by_label = """
        SELECT ?item ?itemLabel
        WHERE {{
            ?item rdfs:label ?itemLabel.
            FILTER(LANG(?itemLabel) = "en").
            FILTER(CONTAINS(LCASE(?itemLabel), {{search_term}})).
        }}
        """


# REQUETE DANS LES PARENTS
query_by_parent = """
        SELECT ?item ?itemLabel
        WHERE {{
            ?item wdt:P31*/wdt:P279* ?parent.
            ?parent rdfs:label ?parentLabel.
            FILTER(LANG(?parentLabel) = "en").
            FILTER(CONTAINS(LCASE(?parentLabel), {{search_term}})).
            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
        }}
        """

# =================================================================================
# AIDES POUR LES REQUETES ULTERIEURES
# =================================================================================

# REQUETE POUR OBTENIR DANS QUELLES PROPRIÉTÉS L'ENTITÉ EST UTILISÉE
query_properties = """
SELECT ?prop WHERE {
  {
    SELECT ?prop 
           WHERE {
      ?item ?prop {{search_term}}.
    }
    GROUP BY ?prop
  }
}
"""

# =================================================================================
# REQUETES THEMATIQUES
# =================================================================================

query_aero_events = """
SELECT ?item ?itemLabel ?lien ?lienLabel ?prop ?propLabel WHERE {
  VALUES ?lien { wd:Q1070669 wd:Q8421 wd:Q765633 wd:Q108284447 }
  ?item ?prop ?lien.
  ?item wdt:P31/wdt:P279 wd:Q1656682.
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "fr,en,[AUTO_LANGUAGE]". }
}
"""

query_events = """
SELECT DISTINCT ?item ?label
WHERE {
  VALUES ?type {
    wd:Q1190554 wd:Q1656682
    }
  ?item wdt:P31 ?evenement .
  ?item wdt:P585 ?date .
  FILTER (?date > "1800-01-01T00:00:00Z"^^xsd:dateTime) .
  ?item rdfs:label ?label .
  FILTER (lang(?label) = "en") .
  
  # Regex sur labels, descriptions ou alias
  FILTER (EXISTS {
    {
      ?item rdfs:label ?text.
      FILTER(REGEX(LCASE(?text), "(aero|aviat|flight|aircraft|airport|\\bplane\\b)", "i"))
    } UNION {
      ?item schema:description ?text.
      FILTER(REGEX(LCASE(?text), "(aero|aviat|flight|aircraft|airport|\\bplane\\b)", "i"))
    } UNION {
      ?item skos:altLabel ?text.
      FILTER(REGEX(LCASE(?text), "(aero|aviat|flight|aircraft|airport|\\bplane\\b)", "i"))
    }
  })
}
"""

query_aero_events_types = """
# Ce code SPARQL extrait une hiérarchie d’éléments Wikidata liés à l’aviation, en marquant pour chaque élément la nature du lien (direct ou hérité), la profondeur dans la hiérarchie, et en affichant les libellés.

SELECT DISTINCT ?item ?itemLabel ?parent ?parentLabel ?depth ?aviationLink WHERE {
  
  # Étape 1 : Sélectionner tous les items qui ont un lien avec l’aviation.
  {
    SELECT DISTINCT ?aviationLinkedItem WHERE {
      # On définit trois types d'évènements de base liés à l’aviation (Q1656682, Q1190554, Q108586636).
      VALUES ?evenement {wd:Q1656682 wd:Q1190554 wd:Q108586636}
      # On cherche les items dont le type (P31) ou un type parent (P279) correspond à ces évènements.
      ?aviationLinkedItem wdt:P31/wdt:P279 ?evenement.
      
      # FILTRE AERONAUTIQUE : Détection de mots-clés dans les libellés, descriptions ou labels alternatifs.
      {
        ?aviationLinkedItem rdfs:label ?label.
        FILTER(REGEX(LCASE(?label), "(aero|aviation|aircraft|flight|aerial|plane|airport|pilot)", "i"))
      } UNION {
        ?aviationLinkedItem schema:description ?desc.
        FILTER(REGEX(LCASE(?desc), "(aero|aviation|aircraft|flight|aerial|plane|airport|pilot)", "i"))
      } UNION {
        ?aviationLinkedItem skos:altLabel ?altLabel.
        FILTER(REGEX(LCASE(?altLabel), "(aero|aviation|aircraft|flight|aerial|plane|airport|pilot)", "i"))
      } UNION {
        # On regarde aussi si l’item est lié, via certaines propriétés, à des entités aéronautiques spécifiques.
        ?aviationLinkedItem (wdt:P31|wdt:P279|wdt:P361|wdt:P527|wdt:P1269) ?aeronauticEntity.
        VALUES ?aeronauticEntity {
          wd:Q8421      # aéronautique
          wd:Q765633    # aviation  
          wd:Q11436     # aircraft
          wd:Q62447     # aérodrome
          wd:Q1248784   # aéroport international
          wd:Q46970     # compagnie aérienne
          wd:Q744913    # accident d'avion
          wd:Q206021    # vol
          wd:Q2876213   # aérospatiale
        }
      }
    }
  }
  
  # Étape 2 : Pour chaque item lié à l’aviation, on remonte toute la hiérarchie de type (P31).
  ?aviationLinkedItem wdt:P31* ?item.
  
  # On garde seulement les items dont le type ultime est Q108586636 (évènement de transport aérien).
  ?item wdt:P31+ wd:Q108586636.
  
  # On récupère le parent direct dans la hiérarchie pour l’affichage.
  OPTIONAL { ?item wdt:P31 ?parent }
  
  # Calcul de la profondeur de l’item dans la hiérarchie (distance au type de base).
  {
    SELECT ?item (COUNT(?intermediate) AS ?depth) WHERE {
      ?item wdt:P31+ ?intermediate.
      ?intermediate wdt:P31* wd:Q108586636.
    }
    GROUP BY ?item
  }
  
  # On limite la profondeur d’analyse à 4 pour éviter des hiérarchies trop longues.
  FILTER(?depth <= 4)
  
  # On marque si le lien aviation est direct (l’item est lui-même identifié comme aviation) ou hérité (par la hiérarchie).
  BIND(IF(?item = ?aviationLinkedItem, "DIRECT", "INHERITED") AS ?aviationLink)
  
  # On ajoute les labels en anglais, français, ou langue auto-détectée.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,fr,[AUTO_LANGUAGE]". }
}
# Tri des résultats par profondeur, type de lien, parent et label.
ORDER BY ?depth ?aviationLink ?parentLabel ?itemLabel
"""


print("✅ Requêtes prêtes !")

✅ Requêtes prêtes !


### Aide à la requête : Lancer une requête individuelle

In [17]:
# EXECUTER UNE REQUÊTE SPARQL DEFINIE AVEC PAGINATION

# =================================================================================
# CHOISIR ICI LA REQUÊTE À EXÉCUTER
# =================================================================================
query = query_properties_one  # Remplacer par la requête souhaitée

# =================================================================================
# FONCTION POUR EXÉCUTER LA REQUÊTE SPARQL AVEC PAGINATION
# =================================================================================
def execute_specific_query(query):
    """
    Exécute une requête SPARQL avec pagination.
    :param query: La requête SPARQL à exécuter.
    :param limit: Nombre de résultats par page (par défaut: MAX_RESULTS).
    :param max_results: Nombre maximum de résultats à récupérer (par défaut: MAX_RESULTS).
    :return: Liste des résultats paginés.
    """
    # Vérifier si la requête est vide
    if not query:
        print("❌ Aucune requête SPARQL fournie.")
        return []
    # Afficher les détails de la requête
    print(f"🔍 Exécution de la requête avec pagination : {query}")
    # Exécuter la requête avec pagination
    return execute_paginated_query(query)

query_results = execute_specific_query(query)

🔍 Exécution de la requête avec pagination : 
    SELECT DISTINCT ?item ?itemLabel ?prop ?itemDescription ?parent ?parentLabel
    WHERE {
      ?item ?prop wd:Q8421 .
      OPTIONAL {
      ?item (wdt:P31|wdt:P279) ?parent .
      ?parent rdfs:label ?parentLabel .
      ?page schema:description ?itemDescription .
      FILTER(LANG(?parentLabel) = "en" || LANG(?parentLabel) = "fr")
      FILTER(LANG(?itemDescription) = "en" || LANG(?itemDescription) = "fr")
      }
      SERVICE wikibase:label { bd:serviceParam wikibase:language "fr,en". }
    }
    
🔍 Exécution de la requête SPARQL :

    SELECT DISTINCT ?item ?itemLabel ?prop ?itemDescription ?parent ?parentLabel
    WHERE {
      ?item ?prop wd:Q8421 .
      OPTIONAL {
      ?item (wdt:P31|wdt:P279) ?parent .
      ?parent rdfs:label ?parentLabel .
      ?page schema:description ?itemDescription .
      FILTER(LANG(?parentLabel) = "en" || LANG(?parentLabel) = "fr")
      FILTER(LANG(?itemDescription) = "en" || LANG(?itemDescription) 

In [ ]:
print(f"✅ Requête exécutée avec succès, {len(query_results)} résultats obtenus.")

# Pour obtenir automatiquement le nom de la variable (ex: "query_aero_events") pointant vers la valeur de query,
# on peut parcourir les variables globales et comparer leur valeur à celle de query.
# Attention : cela ne fonctionne que si la variable est accessible dans le scope global et que la valeur n'est pas modifiée.

def get_query_var_name(query_value):
    for var_name, var_val in globals().items():
        if var_val is query_value:
            return var_name
    return "query"

query_var_name = get_query_var_name(query)
raw_json_filename = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}_RAW_{query_var_name}.json"
raw_json_filepath = os.path.join(output_dir, raw_json_filename)

with open(raw_json_filepath, 'w', encoding='utf-8') as jsonfile:
    json.dump(query_results, jsonfile, ensure_ascii=False, indent=2)
print("=" * 100)
print(f"✅ Résultats sauvegardés dans {raw_json_filepath}")

# 📩 SAUVEGARDE DES RÉSULTATS EN CSV

raw_csv_filename = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}_RAW_{query_var_name}.csv"
raw_csv_filepath = os.path.join(output_dir, raw_csv_filename)
with open(raw_csv_filepath, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = query_results[0].keys() if query_results else []
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for result in query_results:
        writer.writerow({k: v['value'] if isinstance(v, dict) else v for k, v in result.items()})
print(f"✅ Résultats sauvegardés dans {raw_csv_filepath}")


### 1. Aide à la requête : Rechercher une entité par nom 

Cette fonction peut s'utiliser pour retirer tous les termes wikidata qui comprennent une chaîne de caractère
1. Dans leur label
2. Dans un de leurs termes génériques 

In [22]:
# 🔎 RECHERCHE PAR NOM
# =================================================================================


# =================================================================================
# Demande à l'utilisateur le nom de l'entité à rechercher dans Wikidata
# =================================================================================
"""
search_entity_name = input("Entrez le nom de l'entité à rechercher dans wikidata, en anglais (ex: aeronautics) : ").strip()
if not search_entity_name:
    print("❌ Aucun nom d'entité fourni.")
    exit(1)  # Sortir du script si aucun nom n'est fourni
"""


# =================================================================================
# DEMANDE À L'UTILISATEUR LE TYPE DE REQUÊTE À UTILISER
# =================================================================================

# Choix de la requête à utiliser
query_choice = input(
    "Quel type de requête utiliser ?\n"
    "1️⃣ Recherche d'une chaîne dans les labels\n"
    "2️⃣ Requête d'une chaîne dans les parents\n"
    "Entrez le numéro de votre choix (1 ou 2) : "
).strip()
if query_choice not in ["1", "2"]:
    print("❌ Choix invalide. Veuillez entrer 1 ou 2.")
    exit(1)  # Sortir du script si le choix est invalide

if query_choice == "1":
    query_regex = query_by_label
elif query_choice == "2":
    query_regex = query_by_parent

raw_entity_by_name_results = execute_paginated_query(query_regex, ask_term=True)
# def find_entity_by_name(search_entity_name=search_term, query_regex=query_regex):
#     """
#     Demande à l'utilisateur le nom de l'entité à rechercher dans Wikidata
#     :return: Liste des résultats de la recherche
#     """
    
#     print("=" * 100)
#     print(f"🔍 RECHERCHE DES TERMES CONTENANT LA CHAINE DE CARACTERE :  '{search_entity_name}'...")
#     print("=" * 100)
#     print(f"🔍 REQUÊTE ENVOYEE :{query_regex}")
#     print("=" * 100)
#     return execute_paginated_query(query_regex)
# raw_entity_by_name_results = find_entity_by_name()

# =================================================================================
# 📩 SAUVEGARDE DES RÉSULTATS EN JSON
# =================================================================================

def save_raw_results_to_json(raw_entity_by_name_results, search_term):
    """
    Sauvegarde les résultats bruts de la recherche dans un fichier JSON
    :param raw_entity_by_name_results: Résultats bruts de la recherche
    :param search_entity_name: Nom de l'entité recherchée
    """
    # Création du nom de fichier avec la date et l'heure actuelles
    raw_json_filename = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}_RAW_{search_term}.json"
    raw_json_filepath = os.path.join(output_dir, raw_json_filename)

    with open(raw_json_filepath, 'w', encoding='utf-8') as jsonfile:
        json.dump(raw_entity_by_name_results, jsonfile, ensure_ascii=False, indent=2)
    
    print("=" * 100)
    print(f"✅ RESULTATS SAUVEGARDES DANS {raw_json_filepath}")
#save_raw_results_to_json(raw_entity_by_name_results, search_term)

🔍 Choisissez le type de recherche :
1. Recherche par ID (ex: Q42)
2. Recherche par label (ex: Douglas Adams)
🔍 Recherche pour le terme : "douglas adams"
🔍 Requête APRÈS remplacement :

        SELECT ?item ?itemLabel
        WHERE {{
            ?item rdfs:label ?itemLabel.
            FILTER(LANG(?itemLabel) = "en").
            FILTER(CONTAINS(LCASE(?itemLabel), "douglas adams")).
        }}
        
🔍 Exécution de la requête SPARQL :

        SELECT ?item ?itemLabel
        WHERE {{
            ?item rdfs:label ?itemLabel.
            FILTER(LANG(?itemLabel) = "en").
            FILTER(CONTAINS(LCASE(?itemLabel), "douglas adams")).
        }}
        

🔍 Début de la pagination (limit=100)...
🔹 Requête OFFSET 0, LIMIT 100
⚠️  Tentative 1/3 échouée à l'offset 0: The read operation timed out
⚠️  Tentative 2/3 échouée à l'offset 0: The read operation timed out
⚠️  Tentative 3/3 échouée à l'offset 0: The read operation timed out
❌ Requête échouée après 3 tentatives à l'offset 0


### 2. Aide à la requête : Trouver toutes les propriétés dans lesquelles un terme est utilisé

#### Exécuter la requête

In [25]:
# REQUETE PAR PROPRIETE
# =================================================================================

# Exécuter avec demande de terme
query_results = execute_paginated_query(query_properties, ask_term=True)

🔍 Choisissez le type de recherche :
1. Recherche par ID (ex: Q42)
2. Recherche par label (ex: Douglas Adams)
🔍 Recherche pour le terme : Q22719
🔍 Exécution de la requête SPARQL :

SELECT ?prop WHERE {
  {
    SELECT ?prop 
           WHERE {
      ?item ?prop wd:Q22719.
    }
    GROUP BY ?prop
  }
}


🔍 Début de la pagination (limit=100)...
🔹 Requête OFFSET 0, LIMIT 100
✅ Récupéré 43 résultats (total: 43)
⏳ Attente de 3.0s...
🔹 Requête OFFSET 100, LIMIT 100
✅ Fin de la pagination - Aucun résultat supplémentaire.
🎯 Total final récupéré : 43 résultats.


#### Créer un dictionnaire avec index avec la liste de toutes les propriétés extraites

In [23]:
# Extraction simple des IDs de propriété depuis query_results
all_property_ids = [clean_entity_id(prop['prop']['value']).replace('#', ':') for prop in query_results if 'prop' in prop and 'value' in prop['prop']]

print(all_property_ids)


['P2650', 'owl:sameAs', 'P301', 'P31', 'P1535', 'P921', 'P1269', 'P1889', 'P101', 'P279', 'P921', 'P301', 'P31', 'P1535', 'about', 'P1269', 'P1889', 'P279', 'P101', 'P2650', 'P812', 'P101', 'P921', 'P5137', 'P9488', 'P5137', 'P9488']


#### Requêter toutes les pages reliées à l'identifiant recherché

In [ ]:
# Génère une requête SPARQL pour trouver toutes les pages reliées au search_term via les propriétés de all_property_ids

def build_related_pages_query(search_term, property_ids):
    """
    Construit une requête SPARQL pour trouver toutes les pages reliées à search_term via une liste de propriétés.
    :param search_term: ID Wikidata (ex: Q42)
    :param property_ids: liste de propriétés (ex: ['P50', 'P170'])
    :return: requête SPARQL (str)
    """
    # Prépare la liste VALUES pour les propriétés
    # Inclure toutes les propriétés qui contiennent ":" (ex: wdt:, owl:, rdf:, etc.)
    values_block = " ".join(
      (
        f"wdt:{pid}" if pid.startswith("P") else pid
      )
      for pid in property_ids
      if (":" in pid) or pid.startswith("P")
    ).strip()
    # Nettoyer les espaces multiples éventuels
    values_block = " ".join(values_block.split())
    print(values_block)

    query = f"""    SELECT 
  ?item 
  ?itemLabel 
  (GROUP_CONCAT(DISTINCT ?propLabel; separator=", ") AS ?props)
  (COALESCE(?itemDescription_fr, ?itemDescription_en) AS ?itemDescription) # renvoie la première description valide trouvée.
  ?parent1
  (COALESCE(?parent1Label_fr, ?parent1Label_en) AS ?parent1Label)
  ?parent2
  (COALESCE(?parent2Label_fr, ?parent2Label_en) AS ?parent2Label)
WHERE {{
  VALUES ?prop {{ 
    {values_block}
  }}
  ?item ?prop  wd:{search_term} .
  OPTIONAL {{
  ?wd wikibase:directClaim ?prop .
  ?wd rdfs:label ?propLabel .
  FILTER(LANG(?propLabel) = "fr")
  }}

  # Description fr/en selon disponibilité
  OPTIONAL {{ ?item schema:description ?itemDescription_fr . FILTER(LANG(?itemDescription_fr) = "fr") }}
  OPTIONAL {{ ?item schema:description ?itemDescription_en . FILTER(LANG(?itemDescription_en) = "en") }}

  # parent1Label fr/en selon disponibilité
  OPTIONAL {{
    ?item wdt:P31 ?parent1 .
    OPTIONAL {{ ?parent1 rdfs:label ?parent1Label_fr . FILTER(LANG(?parent1Label_fr) = "fr") }}
    OPTIONAL {{ ?parent1 rdfs:label ?parent1Label_en . FILTER(LANG(?parent1Label_en) = "en") }}

    # parent2Label fr/en selon disponibilité
    OPTIONAL {{
      ?item wdt:P279|wdt: ?parent2 .
      OPTIONAL {{ ?parent2 rdfs:label ?parent2Label_fr . FILTER(LANG(?parent2Label_fr) = "fr") }}
      OPTIONAL {{ ?parent2 rdfs:label ?parent2Label_en . FILTER(LANG(?parent2Label_en) = "en") }}
    }}
  }}

  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "fr,en". }}
}}
GROUP BY ?item ?itemLabel ?itemDescription_fr ?itemDescription_en ?parent1Label_fr ?parent1Label_en ?parent2Label_fr ?parent2Label_en ?parent1 ?parent2
"""
    return query

# Exemple d'utilisation :
search_term = ask_query_term()  # à remplacer par votre variable ou input utilisateur
query = build_related_pages_query(search_term, all_property_ids)

# Exécuter la requête pour obtenir les pages liées
related_pages_results = execute_sparql_query(query, use_pagination=False)

# Sauvegarder les résultats dans un fichier JSON
related_pages_json_filename = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}_related_pages_{search_term}.json"
related_pages_json_filepath = os.path.join(output_dir, related_pages_json_filename)
with open(related_pages_json_filepath, 'w', encoding='utf-8') as jsonfile:
    json.dump(related_pages_results, jsonfile, ensure_ascii=False, indent=2)
print(f"✅ Résultats des pages liées sauvegardés dans {related_pages_json_filepath}")

🔍 Choisissez le type de recherche :
1. Recherche par ID (ex: Q42)
2. Recherche par label (ex: Douglas Adams)
wdt:P2650 owl:sameAs wdt:P301 wdt:P31 wdt:P1535 wdt:P921 wdt:P1269 wdt:P1889 wdt:P101 wdt:P279 wdt:P921 wdt:P301 wdt:P31 wdt:P1535 wdt:P1269 wdt:P1889 wdt:P279 wdt:P101 wdt:P2650 wdt:P812 wdt:P101 wdt:P921 wdt:P5137 wdt:P9488 wdt:P5137 wdt:P9488
🔍 Exécution de la requête SPARQL :
    SELECT 
  ?item 
  ?itemLabel 
  (GROUP_CONCAT(DISTINCT ?propLabel; separator=", ") AS ?props)
  (COALESCE(?itemDescription_fr, ?itemDescription_en) AS ?itemDescription) # renvoie la première description valide trouvée.
  ?parent1
  (COALESCE(?parent1Label_fr, ?parent1Label_en) AS ?parent1Label)
  ?parent2
  (COALESCE(?parent2Label_fr, ?parent2Label_en) AS ?parent2Label)
WHERE {
  VALUES ?prop { 
    wdt:P2650 owl:sameAs wdt:P301 wdt:P31 wdt:P1535 wdt:P921 wdt:P1269 wdt:P1889 wdt:P101 wdt:P279 wdt:P921 wdt:P301 wdt:P31 wdt:P1535 wdt:P1269 wdt:P1889 wdt:P279 wdt:P101 wdt:P2650 wdt:P812 wdt:P101 wdt:P

### REQUÊTES THEMATIQUES 

In [ ]:
def build_aeronautics_extraction_queries():
    """Construit les requêtes d'extraction des données"""
    queries = {
        "manufacturers": """
        SELECT DISTINCT ?item 
              
                ?itemLabel
               (COALESCE(?itemDescription_fr, ?itemDescription_en, ?itemDescription_any, "") AS ?itemDescription)
               ?parent
               ?parentLabel
               (GROUP_CONCAT(DISTINCT ?synonym_fr; separator=",") AS ?synonyms_fr)
        WHERE {
          # Tous les constructeurs d'aviation (instances ou sous-classes ou parties)
          ?item wdt:P31*/wdt:P279*/wdt:P361*/wdt:P452*/wdt:P749* wd:Q936518 .
        
          # On cherche le parent immédiat selon différentes relations
            OPTIONAL { ?item wdt:P361 ?partOf . }
            OPTIONAL { ?item wdt:P279 ?subclassOf . }
            OPTIONAL { ?item wdt:P31 ?instanceOf . }
            OPTIONAL { ?item wdt:P452 ?secteur .}
            OPTIONAL { ?item wdt:P176 ?constructeur.}
            OPTIONAL { ?item wdt:P749 ?constructeur.}
            
            BIND(COALESCE(?partOf, ?subclassOf, ?instanceOf, ?secteur, ?constructeur) AS ?parent)
        
          # Description de l'item (fr > en > autre)
          OPTIONAL { ?item schema:description ?itemDescription_fr . FILTER(LANG(?itemDescription_fr) = "fr") }
          OPTIONAL { ?item schema:description ?itemDescription_en . FILTER(LANG(?itemDescription_en) = "en") }
          OPTIONAL { ?item schema:description ?itemDescription_any .
                     FILTER(LANG(?itemDescription_any) != "fr" && LANG(?itemDescription_any) != "en") }

          # Synonymes en français (P1709 est "synonymes exacts")
          OPTIONAL { ?item skos:altLabel ?synonym_fr . FILTER(LANG(?synonym_fr) = "fr") }
        
          SERVICE wikibase:label {
            bd:serviceParam wikibase:language "fr,en,[AUTO_LANGUAGE]"
          }
        }
        GROUP BY ?item ?itemLabel ?itemDescription_fr ?itemDescription_en ?itemDescription_any ?parent ?parentLabel
        """,

        "aircraft_models": """
        SELECT DISTINCT ?item 
               ?itemLabel
               (COALESCE(?itemDescription_fr, ?itemDescription_en, ?itemDescription_any, "") AS ?itemDescription)
               ?parent
               ?parentLabel
               (GROUP_CONCAT(DISTINCT ?synonym_fr; separator=" , ") AS ?synonyms_fr)
        WHERE {
          # Tous les modèles d'avions (instances ou sous-classes ou parties)
          ?item wdt:P31/wdt:P279* wd:Q11436 .
          
        
          # On cherche le parent immédiat selon différentes relations         
            OPTIONAL { ?item wdt:P179 ?series .}
            OPTIONAL { ?item wdt:176 ?constructeur.}
            OPTIONAL { ?item wdt:P31 ?instanceOf . } 
            OPTIONAL { ?item wdt:P361 ?partOf . }
            OPTIONAL { ?item wdt:P279 ?subclassOf . }
                       
            BIND(COALESCE(?partOf, ?subclassOf, ?instanceOf, ?series, ?constructeur) AS ?parent)
        
          # Description de l'item (fr > en > autre)
          OPTIONAL { ?item schema:description ?itemDescription_fr . FILTER(LANG(?itemDescription_fr) = "fr") }
          OPTIONAL { ?item schema:description ?itemDescription_en . FILTER(LANG(?itemDescription_en) = "en") }
          OPTIONAL { ?item schema:description ?itemDescription_any .
                     FILTER(LANG(?itemDescription_any) != "fr" && LANG(?itemDescription_any) != "en") }

          # Synonymes en français
          OPTIONAL { ?item skos:altLabel ?synonym_fr . FILTER(LANG(?synonym_fr) = "fr") }
        
          SERVICE wikibase:label {
            bd:serviceParam wikibase:language "fr,en,[AUTO_LANGUAGE]"
          }
        }
        GROUP BY ?item ?itemLabel ?itemDescription_fr ?itemDescription_en ?itemDescription_any ?parent ?parentLabel
        """,

        "aircraft_components": """
        SELECT DISTINCT ?item 
               ?itemLabel
               (COALESCE(?itemDescription_fr, ?itemDescription_en, ?itemDescription_any, "") AS ?itemDescription)
               ?parent
               ?parentLabel
               (GROUP_CONCAT(DISTINCT ?synonym_fr; separator=" , ") AS ?synonyms_fr)
        WHERE {
          # Tous les équipements d'aviation (instances ou sous-classes ou parties)
          ?item wdt:P31*/wdt:P279*/wdt:P361* wd:Q16693356 .
        
          # On cherche le parent immédiat selon différentes relations
            OPTIONAL { ?item wdt:P361 ?partOf . }
            OPTIONAL { ?item wdt:P279 ?subclassOf . }
            OPTIONAL { ?item wdt:P31 ?instanceOf . }
            OPTIONAL { ?item wdt:P452 ?secteur .}
            OPTIONAL { ?item wdt:P176 ?constructeur.}
            
            BIND(COALESCE(?partOf, ?subclassOf, ?instanceOf, ?secteur, ?constructeur) AS ?parent)
        
          # Description de l'item (fr > en > autre)
          OPTIONAL { ?item schema:description ?itemDescription_fr . FILTER(LANG(?itemDescription_fr) = "fr") }
          OPTIONAL { ?item schema:description ?itemDescription_en . FILTER(LANG(?itemDescription_en) = "en") }
          OPTIONAL { ?item schema:description ?itemDescription_any .
                     FILTER(LANG(?itemDescription_any) != "fr" && LANG(?itemDescription_any) != "en") }

          # Synonymes en français
          OPTIONAL { ?item skos:altLabel ?synonym_fr . FILTER(LANG(?synonym_fr) = "fr") }
        
          SERVICE wikibase:label {
            bd:serviceParam wikibase:language "fr,en,[AUTO_LANGUAGE]"
          }
        }
        GROUP BY ?item ?itemLabel ?itemDescription_fr ?itemDescription_en ?itemDescription_any ?parent ?parentLabel
        """,

        "aeronautic_profession": """
        SELECT DISTINCT ?item 
               ?itemLabel
               (COALESCE(?itemDescription_fr, ?itemDescription_en, ?itemDescription_any, "") AS ?itemDescription)
               ?parent
               ?parentLabel
               (GROUP_CONCAT(DISTINCT ?synonym_fr; separator=" , ") AS ?synonyms_fr)
        WHERE {
        ?item wdt:P425* ?domaine.
        VALUES ?domaine { wd:Q765633 wd:Q906438 wd:Q1434048 wd:Q206814 wd:Q627716 wd:Q221395 wd:Q765633 wd:Q22719}.  
        
          # On cherche le parent immédiat selon différentes relations
            OPTIONAL { ?item wdt:P361 ?partOf . }
            OPTIONAL { ?item wdt:P279 ?subclassOf . }
            OPTIONAL { ?item wdt:P31 ?instanceOf . }
            OPTIONAL { ?item wdt:P452 ?secteur .}
            OPTIONAL { ?item wdt:P176 ?constructeur.}
            OPTIONAL { ?item wdt:P749 ?constructeur.}
            
            BIND(COALESCE(?partOf, ?subclassOf, ?instanceOf, ?secteur, ?constructeur, ?domaine) AS ?parent) # Attention à coalesce pour éviter les doublons
        
          # Description de l'item (fr > en > autre)
          OPTIONAL { ?item schema:description ?itemDescription_fr . FILTER(LANG(?itemDescription_fr) = "fr") }
          OPTIONAL { ?item schema:description ?itemDescription_en . FILTER(LANG(?itemDescription_en) = "en") }
          OPTIONAL { ?item schema:description ?itemDescription_any .
                     FILTER(LANG(?itemDescription_any) != "fr" && LANG(?itemDescription_any) != "en") }

          # Synonymes en français
          OPTIONAL { ?item skos:altLabel ?synonym_fr . FILTER(LANG(?synonym_fr) = "fr") }
        
          SERVICE wikibase:label {
            bd:serviceParam wikibase:language "fr,en,[AUTO_LANGUAGE]"
          }
        }
        GROUP BY ?item ?itemLabel ?itemDescription_fr ?itemDescription_en ?itemDescription_any ?parent ?parentLabel
        """
    }
    return queries

print("✅ Requêtes définies (avec synonymes français)")

## 🔎 Lancer la recherche globale

In [ ]:
# 🛠️ EXTRACTION DES DONNÉES AÉRONAUTIQUES
def extract_all_aeronautics_data():
    """Extrait toutes les données aéronautiques de manière optimisée"""
    print("🏗️ EXTRACTION HIÉRARCHIQUE EXHAUSTIVE")
    print("="*50)
    
    queries = build_aeronautics_extraction_queries()
    all_results = []
    
    for category, query in queries.items():
        print(f"\n🔍 Extraction: {category}")
        
        # ✅ CORRECTION: Utiliser execute_paginated_query au lieu de execute_batch_queries
        results = execute_batch_queries(query)
        
        # Enrichir chaque résultat avec sa catégorie
        for result in results:
            result["source_category"] = category
        
        all_results.extend(results)
        print(f"✅ {len(results)} entités trouvées pour {category}")
    
    print(f"\n🎯 TOTAL: {len(all_results)} entités extraites")
    return all_results

raw_aeronautics_data = extract_all_aeronautics_data()

### Aperçu

In [ ]:
print(raw_aeronautics_data[:5])  # pour afficher un aperçu

json_filename = f"raw.json"
json_filepath = os.path.join(output_dir, json_filename)

with open(json_filepath, 'w', encoding='utf-8') as jsonfile:
    json.dump(raw_aeronautics_data, jsonfile, ensure_ascii=False, indent=2)

## 📁 Export

### Construction du fichier

In [26]:
# 🏗️ CONSTRUCTION DE LA HIÉRARCHIE FINALE

# def get_id_from_uri(uri):
#     # Ex: "http://www.wikidata.org/entity/Q105557" → "Q105557"
#     return uri.split("/")[-1] if uri else ""

def build_final_hierarchy(related_pages_results):
    """Construit la hiérarchie finale avec parents immédiats et catégories"""
    print("🏗️ CONSTRUCTION DE LA HIÉRARCHIE FINALE")
    print("="*45)
    
    # Créer la hiérarchie structurée
    hierarchy = []
    for entry in related_pages_results:
    
        hierarchy.append(
            {
            "ID": clean_entity_id(entry.get("item", {}).get("value", "")),
            "Terme": entry.get("itemLabel", {}).get("value", ""),
            "ID_TG": clean_entity_id(entry.get("parent", {}).get("value", "")),
            "TG": entry.get("parentLabel", {}).get("value", ""),
            "Def": entry.get("itemDescription", {}).get("value", ""),
            "EP": entry.get("synonyms_fr", {}).get("value", ""),
            "TA": entry.get("source_category", {})
        }
        )
    
 
    print(f"✅ Hiérarchie construite: {len(hierarchy)} entrées totales")
    return hierarchy

final_thesaurus = build_final_hierarchy(related_pages_results)
print(f"🎯 Thésaurus final: {len(final_thesaurus)} entrées")

🏗️ CONSTRUCTION DE LA HIÉRARCHIE FINALE
✅ Hiérarchie construite: 233 entrées totales
🎯 Thésaurus final: 233 entrées


### Export du fichier

In [33]:
# 💾 EXPORT FINAL UNIQUE - CSV Occidental European Format (semicolon separated)

import os
import csv
import json
from datetime import datetime

def export_final_thesaurus(thesaurus_data):
    """Exporte le thésaurus final en CSV (point-virgule, format européen) et JSON"""
    print("💾 EXPORT FINAL UNIQUE")
    print("="*25)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. Export CSV - Occidental European (semicolon separator, utf-8-sig BOM)
    csv_filename = f"thesaurus_aeronautique_FINAL_{timestamp}.csv"
    csv_filepath = os.path.join(output_dir, csv_filename)
    
    fieldnames = [
        'ID', 'Terme', 'ID_TG','TG', 'Def', 'EP',
        'TA'
    ]
    
    print(f"📄 Export CSV: {csv_filepath}")
    with open(csv_filepath, 'w', newline='', encoding='utf-8-sig') as csvfile:
        writer = csv.DictWriter(
            csvfile, 
            fieldnames=fieldnames,
            delimiter=';',         # Use semicolon as separator
            quoting=csv.QUOTE_MINIMAL
        )
        writer.writeheader()
        for entry in sorted(thesaurus_data, key=lambda x: x["ID"]):
            # Ensure all values are strings and convert None to empty string
            row = {k: ('' if v is None else str(v)) for k, v in entry.items()}
            # Guarantee all required fields exist in row
            for field in fieldnames:
                row.setdefault(field, '')
            writer.writerow(row)
    
    # 2. Export JSON avec métadonnées
    json_filename = f"thesaurus_aeronautique_FINAL_{timestamp}.json"
    json_filepath = os.path.join(output_dir, json_filename)
    
    stats = analyze_thesaurus_statistics(thesaurus_data)
    
    json_data = {
        "metadata": {
            "title": "Thésaurus Aéronautique Final - Wikidata",
            "description": "Thésaurus exhaustif avec hiérarchie et parents immédiats",
            "version": "1.0-FINAL",
            "created": timestamp,
            "source": "Wikidata SPARQL optimisé",
            "total_entries": len(thesaurus_data),
            "extraction_method": "multi-query_hierarchical",
            "parent_detection": "automatic_wikidata_relations",
            "multilingual_support": True,
            "format": "structured_hierarchical_thesaurus"
        },
        "statistics": stats,
        "data": thesaurus_data
    }
    
    print(f"📄 Export JSON: {json_filepath}")
    with open(json_filepath, 'w', encoding='utf-8') as jsonfile:
        json.dump(json_data, jsonfile, ensure_ascii=False, indent=2)
    
    return csv_filepath, json_filepath, stats

def analyze_thesaurus_statistics(thesaurus_data):
    """Analyse les statistiques du thésaurus final"""
    stats = {
        "total_entries": len(thesaurus_data),
        "categories": {},
        "relation_types": {},
        "languages": {},
        "hierarchy_depth": 0,
        "entries_with_synonyms": 0,
        "entries_with_descriptions": 0
    }
    
    for entry in thesaurus_data:
        # Catégories
        category = entry.get("category", "unknown")
        stats["categories"][category] = stats["categories"].get(category, 0) + 1
        
        # Types de relation
        rel_type = entry.get("relation_type", "unknown")
        stats["relation_types"][rel_type] = stats["relation_types"].get(rel_type, 0) + 1
        
        # Langues
        lang = entry.get("lang", "unknown")
        stats["languages"][lang] = stats["languages"].get(lang, 0) + 1
        
        # Enrichissements
        if entry.get("synonyms"):
            stats["entries_with_synonyms"] += 1
        if entry.get("description"):
            stats["entries_with_descriptions"] += 1
    
    return stats

def display_final_summary(stats, csv_file, json_file):
    """Affiche un résumé final du thésaurus généré"""
    print("\n🎯 RÉSUMÉ FINAL DU THÉSAURUS AÉRONAUTIQUE")
    print("="*50)
    
    print(f"📊 STATISTIQUES GÉNÉRALES:")
    print(f"   • Total d'entrées: {stats['total_entries']}")
    print(f"   • Entrées avec synonymes: {stats['entries_with_synonyms']}")
    print(f"   • Entrées avec descriptions: {stats['entries_with_descriptions']}")
    
    print(f"\n📂 RÉPARTITION PAR CATÉGORIE:")
    for category, count in sorted(stats["categories"].items(), key=lambda x: x[1], reverse=True):
        percentage = (count / stats['total_entries']) * 100
        print(f"   • {category}: {count} ({percentage:.1f}%)")
    
    print(f"\n🔗 TYPES DE RELATIONS:")
    for rel_type, count in sorted(stats["relation_types"].items(), key=lambda x: x[1], reverse=True):
        print(f"   • {rel_type}: {count}")
    
    print(f"\n🌐 LANGUES:")
    for lang, count in stats["languages"].items():
        print(f"   • {lang}: {count}")
    
    print(f"\n📁 FICHIERS GÉNÉRÉS:")
    print(f"   ✅ CSV: {os.path.basename(csv_file)}")
    print(f"   ✅ JSON: {os.path.basename(json_file)}")
    
    print(f"\n🏆 MISSION ACCOMPLIE !")
    print(f" {stats['total_entries']} entrées de thésaurus")


# Export et résumé final
if final_thesaurus:
    csv_file, json_file, statistics = export_final_thesaurus(final_thesaurus)
    display_final_summary(statistics, csv_file, json_file)
else:
    print("❌ Aucun thésaurus à exporter")

💾 EXPORT FINAL UNIQUE
📄 Export CSV: ./output\thesaurus_aeronautique_FINAL_20250708_100339.csv
📄 Export JSON: ./output\thesaurus_aeronautique_FINAL_20250708_100339.json

🎯 RÉSUMÉ FINAL DU THÉSAURUS AÉRONAUTIQUE
📊 STATISTIQUES GÉNÉRALES:
   • Total d'entrées: 233
   • Entrées avec synonymes: 0
   • Entrées avec descriptions: 0

📂 RÉPARTITION PAR CATÉGORIE:
   • unknown: 233 (100.0%)

🔗 TYPES DE RELATIONS:
   • unknown: 233

🌐 LANGUES:
   • unknown: 233

📁 FICHIERS GÉNÉRÉS:
   ✅ CSV: thesaurus_aeronautique_FINAL_20250708_100339.csv
   ✅ JSON: thesaurus_aeronautique_FINAL_20250708_100339.json

🏆 MISSION ACCOMPLIE !
 233 entrées de thésaurus


### Nettoyage des doublons

In [ ]:
# Lire le CSV (remplace 'ton_fichier.csv' par le tien)
df = pd.read_csv(csv_file, sep=';', dtype=str).fillna('')

# Fonction pour concaténer les valeurs uniques (séparées par "|")
def concat_unique(series):
    uniques = set([v.strip() for v in series if v.strip() != ''])
    return " | ".join(sorted(uniques)) if uniques else ''

# Grouper par 'ID', en concaténant les valeurs différentes pour chaque colonne
df_clean = df.groupby('ID', as_index=False).agg(concat_unique)

# Sauvegarder le résultat
df_clean.to_csv(csv_file, sep=';', index=False, encoding='utf-8-sig')

print(f"✅ CSV nettoyé et exporté sous {csv_file}")

